In [3]:
import os
os.chdir('D:\\Gnan\\DA\\KMU\\Prediction-COVID-19')
from utils import *
from model import *
from trainer import *

from torch.optim.adam import Adam
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [4]:
#path = 'Data/SIR_Origin/*.csv'
#dic_files = Load_files.load_files(path, -17, -15)
#path = 'Data/Cleaned_deathCnt/*.csv'
#dic_files = Load_files.load_files(path, -14, -12)
path = 'Data/Variants/alpha_delta/*.csv'
dic_files = Load_files.load_files(path, -18, -16)
data = dic_files['대구']
data

,stdDay,incDec,alpha_variants,delta_variants
0,2021-04-12,16.0,4.16,0.32
1,2021-04-13,9.0,2.34,0.18
2,2021-04-14,11.0,2.86,0.22
3,2021-04-15,18.0,4.68,0.36
4,2021-04-16,4.0,1.04,0.08
...,...,...,...,...
121,2021-08-11,66.0,0.66,62.70
122,2021-08-12,39.0,0.39,37.05
123,2021-08-13,57.0,0.57,54.15
124,2021-08-14,82.0,0.82,77.90


In [5]:
criterion = nn.MSELoss()
lr = 1e-4
patience = 100
hidden_size = 32
num_layers = 4
batch_size = 8
dropout = 0.25

input_size = 3
sequence_length = 60
num_epochs = 10000

#df = Prepare_df.processing(data, 'Date', 'alpha')
df = Prepare_df.processing(data, 'stdDay', 'alpha_variants')

x = df.iloc[:, 0:]
y = df.iloc[:,:1]

ms = MinMaxScaler()
ss = StandardScaler()

ss.fit(x)
ms.fit(y)

x, y, x_ss, y_ms, train_loader, test_loader = Prepare_df.split_data(df, 57, 60, 1, batch_size, 'mto')

model_BiGRU = BiGRU(input_size = input_size,
                      hidden_size = hidden_size,
                      sequence_length = sequence_length,
                      num_layers = num_layers, 
                      dropout = dropout, 
                      device = device).to(device)

model = model_BiGRU

optimizer = Adam(model.parameters(), lr = lr)

torch.Size([57, 60, 3])
torch.Size([57, 1])
torch.Size([7, 60, 3])
torch.Size([7, 1])


In [6]:
loss_list, model,= Trainer.Many_to_One(train_loader, 
                                       test_loader, 
                                       model, 
                                       criterion, 
                                       optimizer, 
                                       num_epochs, 
                                       patience,
                                       device)

RuntimeError: CUDA out of memory. Tried to allocate 66.00 MiB (GPU 0; 4.00 GiB total capacity; 4.67 MiB already allocated; 550.71 MiB free; 22.00 MiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
plt.plot(loss_list)

In [ ]:
label_y, predicted = predict_mto(model, df, x_ss, y_ms)

In [ ]:
print(mape(label_y+1, predicted))
plotting(label_y, predicted, 57)